In [2]:
import pickle
import copy
import pandas as pd
from cowan.cowan.atom_info import ATOM

In [3]:
html_dict = {}
for k, v in ATOM.items():
    html_dict[
        k] = f'https://physics.nist.gov/cgi-bin/ASD/ie.pl?spectra={v[0].strip("*")}&submit=Retrieve+Data&units=1&format=2&order=0&at_num_out=on&sp_name_out=on&ion_charge_out=on&el_name_out=on&seq_out=on&shells_out=on&conf_out=on&level_out=on&ion_conf_out=on&e_out=0&unc_out=on&biblio=on'

In [4]:
# 重新爬取
# data_dict = {}

# 读入已经保存好的文件
with open('atom_data.dat', 'rb') as f:
    data_dict = pickle.load(f)

In [ ]:
for key, html in html_dict.items():
    if key not in data_dict:
        data_dict[key] = pd.read_csv(html)
        print(key)

In [11]:
data_dict[15] = pd.read_csv(html_dict[15])

In [5]:
with open('atom_data.dat', 'wb') as f:
    pickle.dump(data_dict, f)

In [78]:
with open('atom_data.dat', 'rb') as f:
    data = pickle.load(f)
for key, df in data.items():
    if key <= 10:
        temp_df = df[['At. num', 'Ion Charge', 'Ground Shells', 'Ground Config.', 'Ionization Energy (eV)']].__deepcopy__()
        temp_df.columns = ['At. num', 'Ion Charge', 'Ground Shells (a)', 'Ground Config.', 'Ionization Energy (eV)']
    else:
        temp_df = df[['At. num', 'Ion Charge', 'Ground Shells (a)', 'Ground Config.', 'Ionization Energy (eV)']].__deepcopy__()
    for i in range(temp_df.shape[0]):
        for j in range(temp_df.shape[1]):
            if type(temp_df.iloc[i, j]) == str:
                temp = temp_df.iloc[i, j].strip('=')
                temp = temp.strip('"')
                temp_df.iloc[i, j] = temp
    data[key] = temp_df.dropna()

In [79]:
ground_config = {}
ion_energy_ev = {}
for key, value in data.items():
    temp_1_dict = {}
    temp_2_dict = {}
    for i in range(value.shape[0]):
        temp1 = value['Ionization Energy (eV)'][i]
        temp2 = value['Ground Shells (a)'][i]
        if type(temp1) == str:
            temp1 = eval(temp1.strip('[]'))
        temp_1_dict[i] = temp1
        temp_2_dict[i] = temp2
    ion_energy_ev[key] = temp_1_dict
    ground_config[key] = temp_2_dict

In [80]:
new_dict = {}
for key, value in ground_config.items():
    temp_dict = copy.deepcopy(value)  # 每一个原子
    for ion, config in temp_dict.items():
        temp_str = copy.deepcopy(config)  # 每一个离子的组态

        if '[' in temp_str:
            if '[Rn]' in temp_str:  # 86
                temp_str = temp_str.replace('[Rn]', f'{ground_config[86][0]}')
            if '[Hg]' in temp_str:  # 80
                temp_str = temp_str.replace('[Hg]', f'{ground_config[80][0]}')
            if '[Xe]' in temp_str:  # 54
                temp_str = temp_str.replace('[Xe]', f'{ground_config[54][0]}')
            if '[Cd]' in temp_str:  # 48
                temp_str = temp_str.replace('[Cd]', f'{ground_config[48][0]}')
            if '[Kr]' in temp_str:  # 36
                temp_str = temp_str.replace('[Kr]', f'{ground_config[36][0]}')
            if '[Ar]' in temp_str:  # 18
                temp_str = temp_str.replace('[Ar]', f'{ground_config[18][0]}')
            if '[Ne]' in temp_str:  # 10
                temp_str = temp_str.replace('[Ne]', f'{ground_config[10][0]}')

                
        temp_str = temp_str.split('.')
        for i, name in enumerate(temp_str):
            if len(name) == 3:
                temp = list(name)
                temp.insert(2, '0')
                temp_str[i] = ''.join(temp)
            elif len(name) == 2:
                temp = list(name)
                temp.insert(2, '01')
                temp_str[i] = ''.join(temp)
        temp_dict[ion] = ' '.join(temp_str)
    new_dict[key] = temp_dict

In [81]:
for  i in range(1, 101):
    temp_dict = new_dict[i]
    for k, v in temp_dict.items():
        if '[' in v:
            print(i, k, v)

In [82]:
ions_info = {'config':new_dict, 'energy': ion_energy_ev}

In [87]:
ions_info['config'][1]

{0: '1s01'}

In [84]:
import json
json.dump(ions_info, open('ions_info.json', 'w'))

In [86]:
from pathlib import Path

path = Path(__file__)

NameError: name '__file__' is not defined